1. Importación de las librerias y los datasets

In [3]:
import pandas as pd
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as st
import numpy as np
import scipy.stats as stats
import plotly.express as px
from plotly import graph_objects as go
from scipy.stats import levene, mannwhitneyu, shapiro

In [4]:
booking = pd.read_csv(r'Booking.csv')
property = pd.read_csv(r'Property.csv')

2. Comenzamos con el proceso de ETL y revisión de los datos.

In [5]:
booking.head()

,PropertyId,Property_BookingId,BookingCreatedDate,ArrivalDate,DepartureDate,Adults,Children,Infants,Persons,NumNights,Channel,RoomRate,CleaningFee,Revenue,ADR,TouristTax,TotalPaid
0,2883,48819,12/22/2011 18:18,12/28/2011 0:00,1/6/2012 11:00,2,0,0,2,9,RentTheSun,1000.00,0.0,1000.00,111.111111,NaN,0.0
1,2883,48887,12/1/2012,2/4/2012 0:00,2/8/2012 11:00,4,0,1,5,3,RentTheSun,576.21,0.0,576.21,144.052500,NaN,0.0
2,2883,49091,23/02/2012,2/25/2012 0:00,3/3/2012 11:00,3,0,0,3,7,RentTheSun,1522.86,0.0,1522.86,217.551429,NaN,0.0
3,2883,48994,2/4/2012 7:14,3/9/2012 0:00,3/12/2012 11:00,5,0,0,5,3,RentTheSun,623.73,0.0,623.73,207.910000,NaN,0.0
4,2883,49002,2/5/2012 20:46,3/22/2012 0:00,3/28/2012 11:00,5,0,0,5,6,RentTheSun,1245.43,0.0,1245.43,207.571667,NaN,0.0


In [6]:
property.head()

,PropertyId,RealProperty,Capacity,Square,PropertyType,NumBedrooms,ReadyDate
0,2883,Yes,8,110,Apartment,3,12/28/2011 0:00
1,3963,Yes,4,50,Apartment,1,6/13/2013 0:00
2,3964,Yes,4,45,Apa,0,6/15/2013 0:00
3,4138,Yes,5,60,Apartment,2,8/7/2013 0:00
4,4181,Yes,4,45,Apa,0,9/7/2013 0:00


In [7]:
booking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79595 entries, 0 to 79594
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PropertyId          79595 non-null  int64  
 1   Property_BookingId  79595 non-null  int64  
 2   BookingCreatedDate  79595 non-null  object 
 3   ArrivalDate         79595 non-null  object 
 4   DepartureDate       79595 non-null  object 
 5   Adults              79595 non-null  int64  
 6   Children            79595 non-null  int64  
 7   Infants             79595 non-null  int64  
 8   Persons             79595 non-null  int64  
 9   NumNights           79595 non-null  int64  
 10  Channel             79162 non-null  object 
 11  RoomRate            71635 non-null  float64
 12  CleaningFee         79595 non-null  float64
 13  Revenue             71635 non-null  float64
 14  ADR                 79121 non-null  float64
 15  TouristTax          79420 non-null  float64
 16  Tota

Podemos ver que el dataset booking contiene 79595 filas de datos. Tenemos las columnas 'Channel', 'RoomRate', 'Revenue', 'ADR' y 'TouristTax' con datos vacíos o nulos.

In [8]:
property.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PropertyId    344 non-null    int64 
 1   RealProperty  344 non-null    object
 2   Capacity      344 non-null    int64 
 3   Square        344 non-null    int64 
 4   PropertyType  310 non-null    object
 5   NumBedrooms   344 non-null    int64 
 6   ReadyDate     344 non-null    object
dtypes: int64(4), object(3)
memory usage: 18.9+ KB


Podemos ver que el dataset property contiene 344 filas de datos. No tenemos columnas con datos vacios o nulos.

In [9]:
#Revisamos si 'booking' tiene datos duplicados
booking.duplicated().sum()

np.int64(0)

In [10]:
#Revisamos si 'property' tiene datos duplicados
property.duplicated().sum()

np.int64(0)

2. Procederemos a la limpieza y extración de los datos del dataset Booking 

In [11]:
#Cálculo de valores ausentes
print(booking.isna().sum())

PropertyId               0
Property_BookingId       0
BookingCreatedDate       0
ArrivalDate              0
DepartureDate            0
Adults                   0
Children                 0
Infants                  0
Persons                  0
NumNights                0
Channel                433
RoomRate              7960
CleaningFee              0
Revenue               7960
ADR                    474
TouristTax             175
TotalPaid                0
dtype: int64


In [12]:
#Eliminamos todas las filas que contengan valores ausentes
booking.dropna(inplace=True)
print(booking.isna().sum())

PropertyId            0
Property_BookingId    0
BookingCreatedDate    0
ArrivalDate           0
DepartureDate         0
Adults                0
Children              0
Infants               0
Persons               0
NumNights             0
Channel               0
RoomRate              0
CleaningFee           0
Revenue               0
ADR                   0
TouristTax            0
TotalPaid             0
dtype: int64


Ahora el dataset booking no tiene columnas con valores ausentes.

2.1 Procederemos a la conversión de tipo de datos en las columnas las cuales sea necesario. Comenzamos con el dataset booking.

In [13]:
#Convertimos las columnas 'BookingCreatedDate', 'ArrivalDate' y 'DepartureDate' a tipo datetime 
booking['BookingCreatedDate'] = pd.to_datetime(booking['BookingCreatedDate'], errors='coerce', dayfirst=True)
booking['ArrivalDate'] = pd.to_datetime(booking['ArrivalDate'], errors='coerce', dayfirst=True)
booking['DepartureDate'] = pd.to_datetime(booking['DepartureDate'], errors='coerce', dayfirst=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13988\2381731661.py:2: UserWarning: Parsing dates in %m/%d/%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  booking['BookingCreatedDate'] = pd.to_datetime(booking['BookingCreatedDate'], errors='coerce', dayfirst=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13988\2381731661.py:3: UserWarning: Parsing dates in %m/%d/%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  booking['ArrivalDate'] = pd.to_datetime(booking['ArrivalDate'], errors='coerce', dayfirst=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13988\2381731661.py:4: UserWarning: Parsing dates in %m/%d/%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  booking['DepartureDate'] = pd.to_datetime(booking['DepartureDate'], errors='coerce', dayfirst=True)


Ahora procedemos con el dataset property.

In [14]:
#Convertimos la columnas'ReadyDate'a tipo datetime 
property['ReadyDate'] = pd.to_datetime(property['ReadyDate'], errors='coerce', dayfirst=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13988\3122816088.py:2: UserWarning: Parsing dates in %m/%d/%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  property['ReadyDate'] = pd.to_datetime(property['ReadyDate'], errors='coerce', dayfirst=True)


3. Creamos los datasets con un formato parquet para poder importarlos dentro de Google BigQuery.

In [15]:
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
#Creación de files en formato parquet
property.to_parquet("property.parquet", compression='brotli')
booking.to_parquet("booking.parquet", compression='brotli')

In [17]:
pd.read_parquet("property.parquet").dtypes

PropertyId               int64
RealProperty            object
Capacity                 int64
Square                   int64
PropertyType            object
NumBedrooms              int64
ReadyDate       datetime64[ns]
dtype: object

In [18]:
pd.read_csv("dataset_airbnb-scraper-Barcelona.csv").to_parquet("dataset_airbnb-scraper-Barcelona.parquet", compression='brotli')